# Introduction

- In this Notebook we will build and test a simple RAG System using LangChain and Ollama.
- We will use a small Corpus as the New Knowledge we RAG on it.

# Dependencies :

In [ ]:
!pip install -qU langchain langchain_community
!pip install -qU langchain_chroma
!pip install -qU langchain_ollama

- To work with Ollama on Colab, we have an additional library, so we can launch a terminal just like in our local machine !

1. `pciutils` is required by Ollama to detect the GPU type.
2. Installation of Ollama in the runtime instance will be taken care by `curl -fsSL https://ollama.com/install.sh | sh`




In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

# Implimentation :

## Running Ollama
---

In order to use Ollama it needs to run as a service in background parallel to your scripts. Becasue Jupyter Notebooks is built to run code blocks in sequence this make it difficult to run two blocks at the same time. As a workaround we will create a service using subprocess in Python so it doesn't block any cell from running.

Service can be started by command `ollama serve`.

`time.sleep(5)` adds some delay to get the Ollama service up before downloading the model.

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

## Pulling Model
---

Download the LLM model using `ollama pull llama3.2`.

For other models check https://ollama.com/library

In [ ]:
!ollama pull llama3.1:8b

In [ ]:
!ollama list

NAME           ID              SIZE      MODIFIED      
llama3.1:8b    42182419e950    4.7 GB    8 seconds ago    


## Build

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub


In [ ]:
model_name = "llama3.1:8b"

In [ ]:
llm = Ollama(model=model_name,base_url = "http://localhost:11434")

In [ ]:
embed_model = OllamaEmbeddings(
    model=model_name,
    base_url="http://localhost:11434"
)

In [ ]:
text="""
### The Mysterious Visitor of Chefchaouen

In the small village of Chefchaouen, nestled in the heart of the Rif Mountains, lived a young girl named **Lamia**, known for her kindness and radiant smile. She spent her days helping her mother sell traditional rugs at the town’s bustling souk. One day, as she managed the stall alone, a stranger in a white djellaba, named **Omar**, appeared. His accent revealed he was from a distant region, perhaps Marrakech.

Omar admired a rug woven with intricate red and blue patterns. “This design tells an ancient story, doesn’t it?” he asked. Lamia, curious, replied, “Yes, every pattern holds a secret of our culture. But how do you know this?” The man smiled mysteriously and said, “Because I’ve been searching for one like this for a long time.”

Intrigued, Lamia offered him a cup of mint tea. As they talked, he explained that he was looking for a specific rug with a key-shaped motif. According to legend, this rug would unlock the entrance to a hidden cave containing a forgotten treasure. Lamia then remembered an old rug her grandmother had woven years ago, kept safely in their home.

That evening, she showed the rug to Omar. To her astonishment, he exclaimed, “This is the one!” The next day, they set out together into the mountains. After hours of trekking, they arrived at a cave whose entrance was concealed by thorny bushes. Omar unrolled the rug in front of the entrance, and a soft light illuminated the interior. Inside, they discovered chests brimming with ancient jewelry and gold coins.

Omar revealed he wasn’t a treasure hunter but a historian who wanted to preserve this heritage for future generations. Touched by his sincerity, Lamia agreed to help him. Together, they brought some of the treasures back to the village, enriching the community and safeguarding history.

From that day on, Omar and Lamia became inseparable, united by their love for the traditions and mysteries of Morocco.
"""

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 128)
chunks = text_splitter.split_text(text)
vector_store = Chroma.from_texts(chunks, embed_model)

In [ ]:
retriever = vector_store.as_retriever()
chain = create_retrieval_chain(combine_docs_chain = llm , retriever = retriever)

In [ ]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
combine_docs_chain = create_stuff_documents_chain( llm, retrieval_qa_chat_prompt)

In [ ]:
retrival_chain = create_retrieval_chain(retriever,combine_docs_chain)

In [ ]:
response = retrival_chain.invoke({"input":"What is the name of the girl in the Story ?"})
print(response["answer"])

Lamia is the name of the girl in the story.
